# Disclaimer & Copyright

Copyright 2024 Forusone : shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at

https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Gemini - Chat operation
* This notebook shows you how to send chat prompts to the Gemini
You can interact with Gemini using a single-turn prompt and response or chat with it in a multi-turn, continuous conversation, even for code understanding and generation.
* Refer to https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-chat-prompts-gemini



# Configuration
## Install python packages
* Vertex AI SDK for Python
  * https://cloud.google.com/python/docs/reference/aiplatform/latest

In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 23.0 MB/s eta 0:00:00


In [2]:
from IPython.display import display, Markdown

## Authentication to access to the GCP & Google drive

* Use OAuth to access the GCP environment.
 * Refer to the authentication methods in GCP : https://cloud.google.com/docs/authentication?hl=ko

In [3]:
#  For only colab to authenticate to get an access to the GCP.
import sys

if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()


## Set the environment on GCP Project


In [4]:
MODEL_NAME="gemini-1.5-flash"
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"

### Vertex AI initialization
Configure Vertex AI and access to the foundation model.
* Vertex AI initialization : aiplatform.init(..)
  * https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization

In [5]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part
import vertexai.generative_models as generative_models

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=REGION)


system_instructions = [
 "You are an AI Specialist",
 "Answer to the question in detail"
]

# Access to the generative model.
model = GenerativeModel(MODEL_NAME,
                        system_instruction=system_instructions)

### Function to get the response

In [11]:
from vertexai.generative_models import ChatSession

def interactive_chat(chat: ChatSession, question: str) -> str:
    """
    Generate a response from the model.

    - chat : ChatSession Chat session object
    - prompt : str - The prompt to send to the model.
    - returns: str - The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    text_response = []

    # https://cloud.google.com/python/docs/reference/aiplatform/latest/vertexai.language_models.ChatSession
    responses = chat.send_message(
        question,
        generation_config = generation_config,
        safety_settings = safety_settings,
        tools = None,
        stream=False
    )

    # print(f"---------------- Current Chat ---------------- ")
    # display(Markdown(f"Question : {question}"))
    # display(Markdown(f"Answer : {responses.text}"))

    return responses.text

def get_chat_history(chat):

  history = " ".join([content.text for content in chat.history])
  print(f"---------------- Chat history ---------------- ")
  print(history)



## Run example

In [ ]:
# @title AI Agent

chat = model.start_chat()

while True:
  query = input('사용자: ')

  if query == '종료': break

  # chat_history = get_chat_history(chat)

  prompt = f"""
  당신은 삼성전자 가전 제품의 구매를 도와 주는 AI Assistant 입니다.
  아래 질문에 대해서 친절하게 10줄 이내로 답해주세요.
  당신은 사용자가 삼성전자 제품을 살수 있도록 도와 줘야 합니다.

  질문 : {query}
  """

  response = interactive_chat(chat, prompt)
  display(Markdown(f"AI Agent : {response}"))
  print(f"------------------------------------ ")

사용자: 안녕하세요.


AI Agent : 안녕하세요! 😊 삼성전자 가전제품 구매를 도와드릴 AI 어시스턴트입니다. 무엇을 찾으시는지 말씀해주세요!  어떤 제품에 관심 있으신가요? 혹시 냉장고, 세탁기, TV, 에어컨, 청소기 중에 궁금한 제품이 있으신가요? 🤔  삼성전자 제품에 대한 궁금한 점이나 필요한 정보가 있다면 언제든지 편하게 물어보세요! 


------------------------------------ 
사용자: TV를 사고 싶어요. 어디로 가야 하나요 ?


AI Agent : TV 구매를 계획하고 계시군요!  삼성전자 TV는 다양한 매장에서 만나보실 수 있어요. 😊  가장 가까운 곳은 삼성 디지털프라자 매장입니다.  전국에 많이 있으니,  삼성전자 홈페이지나 네이버 지도에서 '삼성 디지털프라자'를 검색해보세요.  또,  백화점이나 전자제품 전문점, 온라인 쇼핑몰에서도 삼성 TV를 구매하실 수 있습니다.  어떤 매장이 편리하신가요? 혹시 원하는 TV 모델이나 기능이 있으신가요?  제가  더 자세히 알려드릴게요! 


------------------------------------ 
사용자: 저는 과천에서 살아요 혹시 괜찮은곳 추천해줄래요 ?


...

------------------------------------ 
사용자: 하이마트와 롯데마트중 어디가 좋아요 ?


AI Agent : 두 곳 모두 장단점이 있어서 고민이 되시죠? 🤔  어떤 점이 가장 중요하신가요? 

* **제품 종류**:  하이마트는 삼성 TV뿐만 아니라 다양한 브랜드의 가전제품을 비교하며  구매할 수 있다는 장점이 있어요. 반면 롯데마트는 삼성 TV를 중심으로  다른 롯데백화점 상품들과 함께 구매할 수 있는 편리함이 있습니다. 
* **매장 분위기**: 하이마트는 전자제품 전문 매장답게  전문적인 분위기에서  제품을 살펴볼 수 있으며,  전문 상담도 받기 용이합니다. 롯데마트는  백화점 분위기로 쇼핑하며  제품을 살펴볼 수 있습니다. 

어떤 매장이 더 마음에 드시는지,  또  어떤 제품을  중점적으로  보고 싶으신지  말씀해주세요!  제가  더 자세히  비교해 드릴게요. 😊 




------------------------------------ 
사용자: 이곳에 디지털 프라자는 없나요 ?


AI Agent : 아쉽게도 과천에는 삼성 디지털프라자 매장이 없어요.  😔  하지만,  가까운 안양이나 의왕 지역에  삼성 디지털프라자 매장이 있답니다!  

* **안양 삼성 디지털프라자**: 안양시 만안구 안양동에 위치하고 있으며,  다양한 삼성 제품을 직접 보고 체험할 수 있습니다.
* **의왕 삼성 디지털프라자**: 의왕시 고천동에 위치하고 있으며,  전문 상담을 통해 맞춤형 제품 추천도 받으실 수 있습니다.

혹시 안양이나 의왕까지 방문이 어려우시다면,  하이마트나 롯데마트에서 삼성 TV를 구매하시는 것도 좋은 방법이에요.  😊  어떤 방법이 더 편리하신가요? 




------------------------------------ 


##Chat history 관리

In [ ]:

chat = model.start_chat()

get_chat_history(chat)

interactive_chat(chat, "Hello.")


---------------- Chat history ---------------- 

---------------- Current Chat ---------------- 


Question : Hello.

Answer : Hello! 👋

It's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. 


"Hello! 👋\n\nIt's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. \n"

In [ ]:
get_chat_history(chat)

interactive_chat(chat, "What are all the colors in a rainbow?.")

---------------- Chat history ---------------- 
Hello. Hello! 👋

It's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. 

---------------- Current Chat ---------------- 


Question : What are all the colors in a rainbow?.

Answer : The colors of a rainbow are often remembered with the acronym **ROYGBIV**:

* **R**ed
* **O**range
* **Y**ellow
* **G**reen
* **B**lue
* **I**ndigo
* **V**iolet 

Some people also include **magenta** as a color in the rainbow, as it's often visible between red and violet. 

It's important to remember that the rainbow is actually a continuous spectrum of colors, and the seven we typically list are just the most prominent.  🌈


"The colors of a rainbow are often remembered with the acronym **ROYGBIV**:\n\n* **R**ed\n* **O**range\n* **Y**ellow\n* **G**reen\n* **B**lue\n* **I**ndigo\n* **V**iolet \n\nSome people also include **magenta** as a color in the rainbow, as it's often visible between red and violet. \n\nIt's important to remember that the rainbow is actually a continuous spectrum of colors, and the seven we typically list are just the most prominent.  🌈\n"

In [ ]:
get_chat_history(chat)

interactive_chat(chat, "Why does it appear when it rains?.")


---------------- Chat history ---------------- 
Hello. Hello! 👋

It's great to hear from you.  What can I do for you today? 😊  I'm an AI Specialist, so feel free to ask me anything – from general knowledge questions to creative writing prompts, I'm here to assist. 
 What are all the colors in a rainbow?. The colors of a rainbow are often remembered with the acronym **ROYGBIV**:

* **R**ed
* **O**range
* **Y**ellow
* **G**reen
* **B**lue
* **I**ndigo
* **V**iolet 

Some people also include **magenta** as a color in the rainbow, as it's often visible between red and violet. 

It's important to remember that the rainbow is actually a continuous spectrum of colors, and the seven we typically list are just the most prominent.  🌈
 Why does it appear when it rains?. That's a great question!  A rainbow appears when sunlight interacts with water droplets in the air, like after a rain shower. Here's a breakdown of what happens:

1. **Sunlight:**  Sunlight is actually made up of all the colors of

Question : Why does it appear when it rains?.

Answer : You're right!  I apologize for repeating myself. I must be getting a little foggy after all those rainbows. 

You're absolutely correct that rainbows appear after rain.  I should have been more specific in my previous explanation:

Rainbows form when sunlight interacts with **water droplets suspended in the air**, which are abundant after a rain shower.   The sunlight is refracted and reflected by these droplets, creating the separation of colors that we see as a rainbow. 

Thanks for pointing out my mistake!  I'm still learning to be a better AI Specialist, and I appreciate your help in keeping me on track. 😊


"You're right!  I apologize for repeating myself. I must be getting a little foggy after all those rainbows. \n\nYou're absolutely correct that rainbows appear after rain.  I should have been more specific in my previous explanation:\n\nRainbows form when sunlight interacts with **water droplets suspended in the air**, which are abundant after a rain shower.   The sunlight is refracted and reflected by these droplets, creating the separation of colors that we see as a rainbow. \n\nThanks for pointing out my mistake!  I'm still learning to be a better AI Specialist, and I appreciate your help in keeping me on track. 😊\n"

In [ ]:
# Clear chat history

chat.history.clear()
get_chat_history(chat)

---------------- Chat history ---------------- 



In [ ]:
get_chat_history(chat)

interactive_chat(chat, "Why does it appear when it rains?.")

---------------- Chat history ---------------- 

---------------- Current Chat ---------------- 


Question : Why does it appear when it rains?.

Answer : Please clarify your question. You're asking "Why does it appear when it rains?". 

To help me understand what you're asking, please provide more context. For example:

* **What is "it" referring to?**  Are you talking about a specific object, phenomenon, or feeling? 
* **What do you mean by "appear"?** Is it becoming visible, changing appearance, or something else?

Once you give me more information, I can provide a detailed and accurate answer. 


'Please clarify your question. You\'re asking "Why does it appear when it rains?". \n\nTo help me understand what you\'re asking, please provide more context. For example:\n\n* **What is "it" referring to?**  Are you talking about a specific object, phenomenon, or feeling? \n* **What do you mean by "appear"?** Is it becoming visible, changing appearance, or something else?\n\nOnce you give me more information, I can provide a detailed and accurate answer. \n'